### Formatting data which is suitable for inputting

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

output = assembler.transform(data)

final_data = output.select('features','churn')

### Train test split and train a logistic regression model

In [ ]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

from pyspark.ml.classification import LogisticRegression

lr_churn = LogisticRegression(labelCol='churn')

fitted_churn_model = lr_churn.fit(train_churn)

### Training summary

In [ ]:
training_sum.predictions.describe().show()

### Evaluate the results in the test set

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

pred_and_labels = fitted_churn_model.evaluate(test_churn)
pred_and_labels.predictions.show()

### AUC

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

auc = churn_eval.evaluate(pred_and_labels.predictions)



### Predict for new data

In [ ]:
final_lr_model = lr_churn.fit(final_data)

new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

test_new_customers = assembler.transform(new_customers)

final_results = final_lr_model.transform(test_new_customers)

final_results.select('Company','prediction').show()